In [1]:
import pandas as pd

In [2]:
impressions_df = pd.read_csv('../datasets/aftonbladet_blocket_impressions_q4.csv')
impressions_df

,id,hostname,installation_id,session_id,beacon_type_chosen,ad_technical_format,channel,chosen_brand,ad_width_chosen,ad_height_chosen,...,demographic,month_iso,first_event_at_iso,placement_ids,placement_ids_chosen,time_in_screen,scroll_speed,total_fixation_duration,part_year,part_month
0,0030a667-1536-488c-8887-41a25dd5749a,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,1180,240,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""57""}",2022-10,2022-10-03T04:48:44.185Z,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10
1,53066e39-a2be-490d-909f-091a4dfac4cb,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,980,240,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""57""}",2022-10,2022-10-03T04:48:44.188Z,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10
2,09816ff6-9cb6-4376-8d45-cd35de9efae3,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,250,480,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""57""}",2022-10,2022-10-03T04:48:44.188Z,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10
3,626cd45d-9306-447f-9b4a-e14aed45e31d,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/criteo.com-iframe-v1,display,display,NaN,980,240,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""57""}",2022-10,2022-10-03T04:48:44.598Z,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10
4,726b7956-4e56-4ab0-841c-922e4d649349,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/criteo.com-iframe-v1,display,display,NaN,180,150,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""57""}",2022-10,2022-10-03T04:48:44.608Z,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792662,c589fad7-52bb-4c48-82fe-78f79ca4035a,blocket.se,cb8ce80f-46fd-4f0e-ac8a-37fee28566de,924ea276-2e58-40ea-a680-456ed6b96e39,ad/adnxs.com-a-img-v1,display,display,adnxs,250,480,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""24""}",2022-01,2022-01-11T18:32:16.499Z,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,NaN,NaN,0,2022,1
792663,4259d5c6-58f6-45cb-8765-452e6529db01,blocket.se,5314c2cc-dc21-46de-a8d7-797328bfcb57,1ccab37c-c553-4a8a-b07e-281e8cbff609,ad/tagid-iframe-v1,display,display,adnxs-simple,980,240,...,"{""country"":"""",""gender"":"""",""age"":""""}",2022-01,2022-01-11T18:25:52.590Z,"{""tag_id"":[""19499521""],""bam_ad_slot"":["""","""",""""]}",NaN,NaN,NaN,0,2022,1
792664,23864762-7d91-4eff-98be-4512ef2a56ea,blocket.se,5314c2cc-dc21-46de-a8d7-797328bfcb57,1ccab37c-c553-4a8a-b07e-281e8cbff609,ad/tagid-iframe-v1,display,display,adnxs,250,480,...,"{""country"":"""",""gender"":"""",""age"":""""}",2022-01,2022-01-11T18:25:52.591Z,"{""tag_id"":[""19499524""],""bam_ad_slot"":["""","""",""""]}",NaN,NaN,NaN,0,2022,1
792665,09afb188-8b90-49a7-8352-bfa78fc4c2c4,blocket.se,cb8ce80f-46fd-4f0e-ac8a-37fee28566de,924ea276-2e58-40ea-a680-456ed6b96e39,ad/tagid-iframe-v1,display,display,adnxs,980,240,...,"{""country"":""Sweden"",""gender"":""Male"",""age"":""24""}",2022-01,2022-01-11T18:31:12.602Z,"{""tag_id"":[""19499516""],""bam_ad_slot"":["""","""",""""]}",NaN,NaN,NaN,0,2022,1


In [3]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def len_pids(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    return len(placement_id["tag_id"]) + len(final_bam_ad_slots)

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_len'] = impressions_df['placement_ids'].apply(len_pids)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [4]:
# filter only impressions with int placement IDs
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
# convert placement ids that can be converted to the int to be int, not string
# type(impressions_df.iloc[0]['pid'])     # check this, right now they're strings
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))
impressions_df.head()

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,hostname,installation_id,session_id,beacon_type_chosen,ad_technical_format,channel,chosen_brand,ad_width_chosen,ad_height_chosen,...,placement_ids,placement_ids_chosen,time_in_screen,scroll_speed,total_fixation_duration,part_year,part_month,pid,pid_len,pid_type
0,0030a667-1536-488c-8887-41a25dd5749a,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,1180,240,...,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10,19499440,1,int
1,53066e39-a2be-490d-909f-091a4dfac4cb,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,980,240,...,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10,19499441,1,int
2,09816ff6-9cb6-4376-8d45-cd35de9efae3,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/tagid-iframe-v1,display,display,NaN,250,480,...,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10,19499420,1,int
5,5b735c98-402c-4fdd-aefc-e54ff032976e,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/anxv,display,display,NaN,180,150,...,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10,19499440,1,int
6,3d70d44c-7f39-4be2-9dbc-e0344fb3aeef,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,de85e076-1f63-456f-87d7-44e204b509b3,ad/anxv,display,display,NaN,180,150,...,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",NaN,0.0,0.0,0,2022,10,19499441,1,int


In [11]:
# groupby and aggregate
grouped_df = impressions_df.groupby(['hostname', 'pid']).apply(
    lambda x: pd.Series(
        {
            'num_impressions' : x['id'].count(),
            'num_in_screens': x.loc[x['is_iab_inview'], 'id'].count()
        }
    )
)
grouped_df

num_impressions  num_in_screens
hostname                  pid                                      
aftonbladet.se            19245836               56              24
                          19499403               28               0
                          19499407            21734           13047
                          19499408            17430             273
                          19499409            11882             621
...                                             ...             ...
spela.aftonbladet.se      14746060                3               0
tramsfrans.aftonbladet.se 14746056                6               2
                          14746059                4               3
                          14746060                5               1
tv.aftonbladet.se         21101514               37              18

[135 rows x 2 columns]

In [12]:
grouped_df['ratio_%'] = round(grouped_df['num_in_screens'] / grouped_df['num_impressions'], 4) * 100

In [13]:
grouped_df

num_impressions  num_in_screens  ratio_%
hostname                  pid                                               
aftonbladet.se            19245836               56              24    42.86
                          19499403               28               0     0.00
                          19499407            21734           13047    60.03
                          19499408            17430             273     1.57
                          19499409            11882             621     5.23
...                                             ...             ...      ...
spela.aftonbladet.se      14746060                3               0     0.00
tramsfrans.aftonbladet.se 14746056                6               2    33.33
                          14746059                4               3    75.00
                          14746060                5               1    20.00
tv.aftonbladet.se         21101514               37              18    48.65

[135 rows x 3 columns]

In [17]:
def get_sample_size(num_impressions):
    if num_impressions <= 10:
        return "low"
    elif num_impressions > 10 and num_impressions <= 20:
        return "medium"
    else:
        return "high"

grouped_df['sample_size'] = grouped_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))
grouped_df = grouped_df.reset_index()

In [18]:
grouped_df['sample_size'].value_counts()

high      103
low        24
medium      8
Name: sample_size, dtype: int64

In [19]:
grouped_df['hostname'].value_counts()

aftonbladet.se               87
blocket.se                   25
malservice.aftonbladet.se     6
bloggar.aftonbladet.se        5
live.aftonbladet.se           3
spela.aftonbladet.se          3
tramsfrans.aftonbladet.se     3
jobb.blocket.se               1
manager.aftonbladet.se        1
tv.aftonbladet.se             1
Name: hostname, dtype: int64

In [23]:
grouped_df.sort_values(by=['hostname', 'num_impressions'], ascending=False, inplace=True)
grouped_df

,hostname,pid,num_impressions,num_in_screens,ratio_%,sample_size
134,tv.aftonbladet.se,21101514,37,18,48.65,high
131,tramsfrans.aftonbladet.se,14746056,6,2,33.33,low
133,tramsfrans.aftonbladet.se,14746060,5,1,20.00,low
132,tramsfrans.aftonbladet.se,14746059,4,3,75.00,low
130,spela.aftonbladet.se,14746060,3,0,0.00,low
...,...,...,...,...,...,...
76,aftonbladet.se,23373575,3,2,66.67,low
54,aftonbladet.se,19499487,2,0,0.00,low
73,aftonbladet.se,21067200,2,0,0.00,low
71,aftonbladet.se,21067197,2,0,0.00,low


In [26]:
result_df = grouped_df[['hostname', 'pid', 'ratio_%', 'sample_size']]
result_df.rename(columns={'pid': 'placement_id', 'ratio_%': 'in_screen_ratio_%'}, inplace=True)
result_df

/home/markopejic/clear/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,hostname,placement_id,in_screen_ratio_%,sample_size
134,tv.aftonbladet.se,21101514,48.65,high
131,tramsfrans.aftonbladet.se,14746056,33.33,low
133,tramsfrans.aftonbladet.se,14746060,20.00,low
132,tramsfrans.aftonbladet.se,14746059,75.00,low
130,spela.aftonbladet.se,14746060,0.00,low
...,...,...,...,...
76,aftonbladet.se,23373575,66.67,low
54,aftonbladet.se,19499487,0.00,low
73,aftonbladet.se,21067200,0.00,low
71,aftonbladet.se,21067197,0.00,low


In [27]:
result_df.to_csv('schibsted_in_screen_report.csv', index=False)